In [1]:
# https://userguide.mdanalysis.org/stable/examples/analysis/trajectory_similarity/clustering_ensemble_similarity.html
import numpy as np
import matplotlib.pyplot as plt

import MDAnalysis as mda
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm
from MDAnalysis.analysis import align


/home/user/anaconda3/envs/Xponge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/anaconda3/envs/Xponge/lib/python3.11/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [ ]:
import os
import glob

clustered_pdb_list = []
for iter in range(0,10,1):
    path = 'PROTAC_8QW7_results/FFT_cluster_' + str(iter) + '/pred'

    for filename in glob.glob(path + '/' + '*.pdb'):
        clustered_pdb_list.append(filename)

print(clustered_pdb_list)
print(len(clustered_pdb_list))

clustered_pdb_list = ['./' + i for i in clustered_pdb_list]
print(clustered_pdb_list)
print(len(clustered_pdb_list))


['PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_3_10.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_10_8.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_10_35.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_4_19.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_6_6.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_6_27.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_3_47.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_1_40.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_8_27.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_4_22.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_1_12.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_2_16.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_1_28.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_3_28.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluster_0/pred/pred_8_22.pdb', 'PROTAC_8QW7_results_FFTtry4/FFT_cluste

In [3]:

clustered_traj = mda.Universe(clustered_pdb_list[0], clustered_pdb_list)

print(len(clustered_traj.trajectory))

ref_for_traj = mda.Universe('8QW7_AB.pdb', '8QW7_AB.pdb')

ref_for_E3 = mda.Universe('8qw7_B.pdb', '8qw7_B.pdb')


/home/user/anaconda3/envs/Xponge/lib/python3.11/site-packages/MDAnalysis/coordinates/base.py:725: UserWarning: Reader has no dt information, set to 1.0 ps
  return self.ts.dt


6642


In [4]:
# E3 is receptor, and POI is ligand.
ref_for_E3_com = ref_for_E3.atoms.center(weights=None)
print(ref_for_E3_com)
ref_E3_CA = ref_for_E3.select_atoms('name CA')
print(len(ref_E3_CA.atoms))

print(ref_E3_CA.positions[0:3,:])
ref_for_E3.atoms.translate(-ref_for_E3_com)
print(ref_E3_CA.positions[0:3,:])

#ref_for_E3.atoms.write("E3_ref_translated.pdb")


traj_E3_CA = clustered_traj.select_atoms('name CA and chainID B')
print(len(traj_E3_CA.atoms))



[18.01563692 22.82902864 15.16912048]
148
[[23.579 42.334  7.923]
 [21.98  38.917  8.227]
 [18.264 38.561  8.724]]
[[ 5.5633636  19.50497    -7.2461205 ]
 [ 3.9643626  16.08797    -6.94212   ]
 [ 0.24836302 15.731972   -6.4451203 ]]
148


In [5]:


with mda.Writer('aligned_all.pdb', clustered_traj.atoms.n_atoms) as w:
    for ts in clustered_traj.trajectory:

        print(ts.frame)
        traj_E3_CA_com = traj_E3_CA.atoms.center(weights=None)
        clustered_traj.atoms.translate(-traj_E3_CA_com)
        if ts.frame == 0:
            clustered_traj.atoms.write("traj_translated.pdb")

        R = align.rotation_matrix(traj_E3_CA.positions,
                                  ref_E3_CA.positions)[0]
        print(R)

        clustered_traj.atoms.rotate(R)
        if ts.frame == 0:
            clustered_traj.atoms.write("traj_rotated.pdb")

        clustered_traj.atoms.translate(ref_for_E3_com)
        w.write(clustered_traj.atoms)

        #break


0
[[-0.23881109 -0.95692908 -0.16509394]
 [ 0.58806075 -0.27780787  0.75960999]
 [-0.77275728  0.08431802  0.62907603]]
1
[[-0.64943652 -0.72421819 -0.23181938]
 [ 0.21613755 -0.46809076  0.85684047]
 [-0.72905197  0.50635862  0.46052598]]
2
[[-0.93081957 -0.19257161 -0.31063018]
 [ 0.05143512 -0.91049002  0.41031981]
 [-0.36184163  0.36595641  0.85740687]]


/home/user/anaconda3/envs/Xponge/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/user/anaconda3/envs/Xponge/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


3
[[-0.71493665 -0.63426706 -0.2942293 ]
 [ 0.26146581 -0.63281323  0.72882306]
 [-0.64846066  0.44413141  0.61826052]]
4
[[-0.41547381 -0.87881216 -0.23467145]
 [ 0.55579224 -0.44950084  0.69931679]
 [-0.72005312  0.16011924  0.67519281]]
5
[[-0.00627726 -0.98418077 -0.17705595]
 [ 0.78962501 -0.11351677  0.60299775]
 [-0.61355761 -0.13602264  0.77784632]]
6
[[-0.09718335 -0.9837833  -0.15075085]
 [ 0.60870837 -0.17858785  0.77303331]
 [-0.78741953 -0.01663733  0.61619289]]
7
[[-0.3626681  -0.90049604 -0.23995567]
 [ 0.42486236 -0.38893591  0.81744775]
 [-0.82943584  0.19451409  0.52364153]]
8
[[-0.86678228 -0.40555222 -0.29019974]
 [ 0.26171008 -0.86528414  0.42754086]
 [-0.42449537  0.29463665  0.85615006]]
9
[[-0.72170628 -0.61635229 -0.31503953]
 [ 0.16853127 -0.59789401  0.78365806]
 [-0.67136969  0.51247693  0.5353785 ]]
10
[[-0.54237587 -0.80220443 -0.24959259]
 [ 0.31895884 -0.47145995  0.82218658]
 [-0.77723463  0.3663244   0.5115787 ]]
11
[[-0.52816575 -0.82660364 -0.1943382